In [1]:
import os

In [2]:
%pwd

'd:\\Loan_Approval_Prediction\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\Loan_Approval_Prediction'

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [12]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [13]:
def clean_data(st):
    st = st.strip()
    return st

In [14]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        # Load raw data
        df = pd.read_csv(self.config.data_path)
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
        # -------------------------
        # ✅ Data Preprocessing
        # -------------------------
        df.education = df.education.apply(clean_data)
        df.self_employed = df.self_employed.apply(clean_data)
        df.loan_status = df.loan_status.apply(clean_data)
        df['education'] = df['education'].astype('category')
        df['self_employed'] = df['self_employed'].astype('category')
        df['loan_status'] = df['loan_status'].astype('category')
         # Encode categorical variables
        df.self_employed = df.self_employed.replace(['No', 'Yes'],[0,1])
        df.loan_status = df.loan_status.replace(['Approved', 'Rejected'],[1,0])
        df['education'] = df['education'].replace(['Graduate', 'Not Graduate'],[1,0])
        # Feature engineering: create total_assets
        df['total_assets'] = (
            df['residential_assets_value'] +
            df['commercial_assets_value'] +
            df['luxury_assets_value'] +
            df['bank_asset_value']
        )

        # Drop unnecessary columns
        df = df.drop(columns=[
            'loan_id',
            'residential_assets_value',
            'commercial_assets_value',
            'luxury_assets_value',
            'bank_asset_value'
        ])

        # -------------------------
        # ✅ Train-test split
        # -------------------------
        from sklearn.model_selection import train_test_split
        train, test = train_test_split(df, test_size=0.25, random_state=42)

        # Save to artifacts directory
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Encoded & engineered data, then split into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        print(df.columns)
        print(df.head(4))


In [15]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-09-03 22:28:09,597: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-03 22:28:09,599: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-03 22:28:09,602: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-03 22:28:09,603: INFO: common: created directory at: artifacts]
[2025-09-03 22:28:09,604: INFO: common: created directory at: artifacts/data_transformation]
[2025-09-03 22:28:09,669: INFO: 2684419802: Encoded & engineered data, then split into training and test sets]
[2025-09-03 22:28:09,670: INFO: 2684419802: (3201, 9)]
[2025-09-03 22:28:09,672: INFO: 2684419802: (1068, 9)]
(3201, 9)
(1068, 9)
Index(['no_of_dependents', 'education', 'self_employed', 'income_annum',
       'loan_amount', 'loan_term', 'cibil_score', 'loan_status',
       'total_assets'],
      dtype='object')
   no_of_dependents education self_employed  income_annum  loan_amount  \
0                 2         1             0       9600000     29900000   
1 